# Imports


In [2]:
from scraper import Scraper
from schema import Schema
import utils
from tqdm import tqdm
from dotenv import dotenv_values

config = dotenv_values("../.env")

In [3]:
from pydantic import BaseModel, Field, field_validator
from typing import List

import openai
import json
import time
from datetime import datetime

In [4]:
scraper = Scraper(config["URL"])
table_rows = scraper.get_table()

# Pydantic schemas


In [5]:
class MovieItem(Schema):
    """
    class representing the entry of the highest grossing movie at the box office in any week of 2023 along the grossing amount and some remarks
    """

    weekend_end_date: str = Field(
        ...,
        alias="Weekend end date",
        description="Stores the last date of the week in the year 2023",
    )
    film: str = Field(..., alias="Film", description="Name of the movie")
    gross: str = Field(
        ..., alias="Gross", description="Gross income of the movie in the given weekend"
    )
    notes: str = Field(..., alias="Notes", description="Some notes about the movie")


class BoxOffice(Schema):
    """Class representing the list of the highest grossing movies per weekend of 2023"""

    items: List[MovieItem] = Field(
        ..., description="List of the highest grossing movies each weekend of 2023"
    )

# Query OpenAI API


In [14]:
def html_to_json_ai(
    data: str, model: str, api_key: str, url: str | None = None
) -> dict:
    """
    Function to convert the given raw HTML string into structured JSON object following a predefined schema
    by calling llm using api keys

    Parameters:
    - data (str):       the raw string containing the html input
    - model (str):      the name of the model endpoint to be called
    - api_key (str):    the api key used for calling the server endpoint
    - url (str | None): the base url used to make the query
                        default: None

    Output:
    json dumped from the BoxOffice object after parsing the model response in the needed format
    """
    client = openai.OpenAI(api_key=api_key, base_url=url)
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": "You are a world class algorithm to convert html to structured data",
            },
            {
                "role": "user",
                "content": f"Convert the following HTML string to structured data as per given format",
            },
            {"role": "user", "content": f"HTML input: ```{data}```"},
            {
                "role": "user",
                "content": "Tips: Make sure to pay attention to the attributes of the HTML tags, especialy the rowspan attributes",
            },
        ],
        temperature=0.2,
        functions=[BoxOffice.custom_schema],
        function_call={"name": BoxOffice.custom_schema["name"]},
    )
    input_tokens = completion.usage.prompt_tokens
    output_tokens = completion.usage.completion_tokens

    input_charge = (input_tokens / 1000) * 0.0010
    output_charge = (output_tokens / 1000) * 0.0020

    print(
        f"Token counts:\nInput tokens = {input_tokens}\nOutput tokens = {output_tokens}\nTotal tokens = {input_tokens + output_tokens}\n"
    )
    # Charges as per OpenAI pricing
    print(
        f"Charge:\nInput charge = ${input_charge}\nOutput charge = ${output_charge}\nTotal charge ${input_charge + output_charge}"
    )
    return BoxOffice.from_response(completion).model_dump_json()

# Results with `GPT-3.5-turbo`


In [15]:
html_chunks = utils.html_chunker(table_rows, chunk_size=4)
html_input = utils.html_to_str(html_chunks[0])
output = html_to_json_ai(
    html_input, model="gpt-3.5-turbo", api_key=config["OPENAI_API_KEY"]
)

Token counts:
Input tokens = 637
Output tokens = 426
Total tokens = 1063

Charge:
Input charge = $0.000637
Output charge = $0.000852
Total charge $0.0014889999999999999


In [16]:
print(json.dumps(json.loads(output), indent=4))

{
    "items": [
        {
            "weekend_end_date": "January 8, 2023",
            "film": "Avatar: The Way of Water",
            "gross": "$45,838,986",
            "notes": "Black Panther: Wakanda Forever and Avatar: The Way of Water became the first two films to consecutively top the box office for four consecutive weekends each since The Hunger Games: Mockingjay \u2013 Part 2 and Star Wars: The Force Awakens in 2015 and 2016."
        },
        {
            "weekend_end_date": "January 15, 2023",
            "film": "Avatar: The Way of Water",
            "gross": "$32,824,684",
            "notes": "Black Panther: Wakanda Forever and Avatar: The Way of Water became the first two films to consecutively top the box office for five consecutive weekends each since Stakeout and Fatal Attraction in 1987."
        },
        {
            "weekend_end_date": "January 22, 2023",
            "film": "Avatar: The Way of Water",
            "gross": "$20,133,106",
            "note

## Total output


In [6]:
html_chunks = utils.html_chunker(table_rows)
for i, chunk in enumerate(html_chunks):
    html_input = utils.html_to_str(chunk)
    print(f"\nChunk {i}")
    output = html_to_json_ai(
        html_input, model="gpt-3.5-turbo", api_key=config["OPENAI_API_KEY"]
    )
    utils.save_json(output, filepath=f"../chunk-{i}.json")
    time.sleep(30)


Chunk 0
Token counts:
Input tokens = 997
Output tokens = 769
Total tokens = 1766

Charge:
Input charge = $0.000997
Output charge = $0.001538
Total charge $0.002535

Chunk 1
Token counts:
Input tokens = 1012
Output tokens = 785
Total tokens = 1797

Charge:
Input charge = $0.001012
Output charge = $0.00157
Total charge $0.002582

Chunk 2
Token counts:
Input tokens = 935
Output tokens = 670
Total tokens = 1605

Charge:
Input charge = $0.0009350000000000001
Output charge = $0.00134
Total charge $0.002275

Chunk 3
Token counts:
Input tokens = 804
Output tokens = 558
Total tokens = 1362

Charge:
Input charge = $0.000804
Output charge = $0.001116
Total charge $0.00192

Chunk 4
Token counts:
Input tokens = 887
Output tokens = 655
Total tokens = 1542

Charge:
Input charge = $0.000887
Output charge = $0.0013100000000000002
Total charge $0.002197


In [8]:
chunk_filepaths = [f"../chunk-{i}.json" for i in range(5)]
combined_output = {"items": []}
for fp in tqdm(chunk_filepaths):
    f = open(fp, "r")
    chunk_output = json.load(f)
    combined_output["items"].extend(chunk_output["items"])

100%|██████████| 5/5 [00:00<00:00, 592.82it/s]


In [10]:
with open("../combined-output.json", "w") as f:
    json.dump(combined_output, f, indent=4)

## Querying by date


Say for example i want only those movies that were the highest grossing in November


In [28]:
def html_to_json_ai(
    data: str, model: str, api_key: str, url: str | None = None
) -> dict:
    """
    Function to convert the given raw HTML string into structured JSON object following a predefined schema
    by calling llm using api keys

    Parameters:
    - data (str):       the raw string containing the html input
    - model (str):      the name of the model endpoint to be called
    - api_key (str):    the api key used for calling the server endpoint
    - url (str | None): the base url used to make the query
                        default: None

    Output:
    json dumped from the BoxOffice object after parsing the model response in the needed format
    """
    client = openai.OpenAI(api_key=api_key, base_url=url)
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": "You are a world class algorithm to convert html to structured data",
            },
            {
                "role": "user",
                "content": f"Convert rows in given HTML corresponding to only the MONTH of NOVEMBER to the specified JSON",
            },
            {"role": "user", "content": f"HTML input: ```{data}```"},
            # {
            #     "role": "user",
            #     "content": "Tips: Make sure to pay attention to the attributes of the HTML tags, especialy the rowspan attributes",
            # },
            # {"role": "user", "content": "Tips: pay attention to the dates"},
        ],
        temperature=0.2,
        functions=[BoxOffice.custom_schema],
        function_call={"name": BoxOffice.custom_schema["name"]},
    )
    input_tokens = completion.usage.prompt_tokens
    output_tokens = completion.usage.completion_tokens

    input_charge = (input_tokens / 1000) * 0.0010
    output_charge = (output_tokens / 1000) * 0.0020

    print(
        f"Token counts:\nInput tokens = {input_tokens}\nOutput tokens = {output_tokens}\nTotal tokens = {input_tokens + output_tokens}\n"
    )
    # Charges as per OpenAI pricing
    print(
        f"Charge:\nInput charge = ${input_charge}\nOutput charge = ${output_charge}\nTotal charge ${input_charge + output_charge}"
    )
    return BoxOffice.from_response(completion).model_dump_json()

In [29]:
html_input = utils.html_to_str(table_rows)

In [30]:
output = html_to_json_ai(
    html_input, model="gpt-3.5-turbo", api_key=config["OPENAI_API_KEY"]
)

Token counts:
Input tokens = 3697
Output tokens = 215
Total tokens = 3912

Charge:
Input charge = $0.003697
Output charge = $0.00043
Total charge $0.0041270000000000005


We are dangerously close to the maximum token length of 4096


In [31]:
print(json.dumps(json.loads(output), indent=4))

{
    "items": [
        {
            "weekend_end_date": "November 5, 2023",
            "film": "Five Nights at Freddy's",
            "gross": "$19,001,870",
            "notes": ""
        },
        {
            "weekend_end_date": "November 12, 2023",
            "film": "The Marvels",
            "gross": "$46,110,859",
            "notes": ""
        },
        {
            "weekend_end_date": "November 19, 2023",
            "film": "The Hunger Games: The Ballad of Songbirds & Snakes",
            "gross": "$44,607,143",
            "notes": ""
        },
        {
            "weekend_end_date": "November 26, 2023",
            "film": "The Hunger Games: The Ballad of Songbirds & Snakes",
            "gross": "$29,042,517",
            "notes": ""
        }
    ]
}


# Trying with `mistral` and `llama` from [Anyscale](https://www.anyscale.com/)


In [17]:
def html_to_json_ai(
    data: str, model: str, api_key: str, url: str | None = None
) -> dict:
    """
    Function to convert the given raw HTML string into structured JSON object following a predefined schema
    by calling llm using api keys

    Parameters:
    - data (str):       the raw string containing the html input
    - model (str):      the name of the model endpoint to be called
    - api_key (str):    the api key used for calling the server endpoint
    - url (str | None): the base url used to make the query
                        default: None

    Output:
    json dumped from the BoxOffice object after parsing the model response in the needed format
    """
    client = openai.OpenAI(api_key=api_key, base_url=url)
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": "You are a world class algorithm to convert html to structured data",
            },
            {
                "role": "user",
                "content": f"Convert rows in given HTML to the specified JSON for only the month of February.",
            },
            {"role": "user", "content": f"HTML input: ```{data}```"},
            # {
            #     "role": "user",
            #     "content": "Tips: Make sure to pay attention to the attributes of the HTML tags, especialy the rowspan attributes",
            # },
            # {"role": "user", "content": "Tips: pay attention to the dates"},
        ],
        temperature=0.2,
        response_format={
            "type": "json_object",
            "schema": BoxOffice.model_json_schema(),
        },
    )
    try:
        return json.loads(completion.choices[0].message.content)
    except:
        print(completion.choices[0].message.content)

First with mistral for a subset of rows


In [9]:
html_chunks = utils.html_chunker(table_rows, chunk_size=4)
html_input = utils.html_to_str(html_chunks[0])
output = html_to_json_ai(
    html_input,
    model="mistralai/Mistral-7B-Instruct-v0.1",
    api_key=config["ANYSCALE_API_KEY"],
    url="https://api.endpoints.anyscale.com/v1",
)
print(json.dumps(output, indent=4))

{
    "items": [
        {
            "Weekend end date": "January 8, 2023",
            "Film": "Avatar: The Way of Water",
            "Gross": "$45,838,986",
            "Notes": "Black Panther: Wakanda Forever and Avatar: The Way of Water became the first two films to consecutively top the box office for four consecutive weekends each since The Hunger Games: Mockingjay \u2013 Part 2 and Star Wars: The Force Awakens in 2015 and 2016."
        },
        {
            "Weekend end date": "January 15, 2023",
            "Film": "Avatar: The Way of Water",
            "Gross": "$32,824,684",
            "Notes": "Black Panther: Wakanda Forever and Avatar: The Way of Water became the first two films to consecutively top the box office for five consecutive weekends each since Stakeout and Fatal Attraction in 1987."
        },
        {
            "Weekend end date": "January 22, 2023",
            "Film": "Avatar: The Way of Water",
            "Gross": "$20,133,106",
            "Note

Then with llama to demo querying by date


In [18]:
html_chunks = utils.html_chunker(table_rows, chunk_size=20)
html_input = utils.html_to_str(html_chunks[0])
output = html_to_json_ai(
    html_input,
    model="meta-llama/Llama-2-13b-chat-hf",
    api_key=config["ANYSCALE_API_KEY"],
    url="https://api.endpoints.anyscale.com/v1",
)
print(json.dumps(output, indent=4))

  Sure! Here is the JSON output for only the month of February:
```json
[
  {
    "week": "February 5",
    "gross": "14,127,170",
    "film": "Knock at the Cabin"
  },
  {
    "week": "February 12",
    "gross": "8,305,317",
    "film": "Magic Mike's Last Dance"
  },
  {
    "week": "February 19",
    "gross": "106,109,650",
    "film": "Ant-Man and the Wasp: Quantumania"
  },
  {
    "week": "February 26",
    "gross": "31,964,803",
    "film": "Ant-Man and the Wasp: Quantumania"
  }
]
```
Note that I have only included the rows that correspond to the month of February. The other rows have been omitted.
null


**Note**: llama tends to return additional text along with the actual json output, and it adjusts the keys in the json. Mistral gives the exact json as output


# Trying with `mistral` and `llama` with prompts


In [24]:
json_format = """```json
{
    "items": [
        {
            "weekend_end_date": "" // Stores the last date of the week in the year 2023
            "film": "", // Name of the movie
            "gross": "", // Gross income of the movie in the given weekend
            "notes": "", // Some notes about the movie
        }.
        // repeat each item of the row as above format
    ]
}
```
"""

In [38]:
def html_to_json_ai(
    data: str, model: str, json_format: str, api_key: str, url: str | None = None
) -> dict:
    """
    Function to convert the given raw HTML string into structured JSON object following a predefined schema
    by calling llm using api keys

    Parameters:
    - data (str):       the raw string containing the html input
    - model (str):      the name of the model endpoint to be called
    - api_key (str):    the api key used for calling the server endpoint
    - url (str | None): the base url used to make the query
                        default: None

    Output:
    json dumped from the BoxOffice object after parsing the model response in the needed format
    """
    client = openai.OpenAI(api_key=api_key, base_url=url)
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": "You are a world class algorithm to convert html to json as per specified format",
            },
            {
                "role": "user",
                "content": f"Convert rows in given HTML to the specified JSON",
            },
            {"role": "user", "content": f"HTML input: ```{data}```"},
            {"role": "user", "content": f"JSON format {json_format}"}
            # {
            #     "role": "user",
            #     "content": "Tips: Make sure to pay attention to the attributes of the HTML tags, especialy the rowspan attributes",
            # },
            # {"role": "user", "content": "Tips: pay attention to the dates"},
        ],
        max_tokens=2048,
    )
    return completion

For some reason mistral models on together are throwing some internal server error. Other models give varying degree of outputs. The one below is a very peculiar case of outputs.


In [39]:
html_chunks = utils.html_chunker(table_rows, chunk_size=4)
html_input = utils.html_to_str(html_chunks[0])
output = html_to_json_ai(
    html_input,
    model="togethercomputer/StripedHyena-Nous-7B",
    json_format=json_format,
    api_key=config["TOGETHERAI_API_KEY"],
    url="https://api.together.xyz",
)

In [40]:
print(output.choices[0].message.content)

### Response:
Here's the Python code to convert the given HTML table to the specified JSON format:
```python
import re
from bs4 import BeautifulSoup
# HTML input
html_input = """
<tr>
<th>.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}#
</th>
<th>Weekend end date</th>
<th>Film</th>
<th>Gross</th>
<th>Notes</th>
<th class="unsortable">Ref.
</th></tr>

<tr>
<td>1</td>
<td>January 8, 2023</td>
<td rowspan="4">Avatar: The Way of Water</td>
<td>$45,838,986</td>
<td>Black Panther: Wakanda Forever and Avatar: The Way of Water became the first two films to consecutively top the box office for four consecutive weekends each since The Hunger Games: Mockingjay – Part 2 and Star Wars: The Force Awakens in 2015 and 2016.</td>
<td>[2]</td></tr>

<tr>
<td>2</td>
<td>January 15, 2023</td>
<td>$32,824,684</td>
<td>Black Panther: Wakanda Forever and Avatar: The Way of Water became the first two films to consecutively top the box office for five consecutive weekends each since Sta

In [41]:
html_chunks = utils.html_chunker(table_rows, chunk_size=4)
html_input = utils.html_to_str(html_chunks[0])
output = html_to_json_ai(
    html_input,
    model="togethercomputer/Llama-2-7B-32K-Instruct",
    json_format=json_format,
    api_key=config["TOGETHERAI_API_KEY"],
    url="https://api.together.xyz",
)

InternalServerError: Error code: 500 - {'error': '(unknown path)\n  Error: Unable to call `raise_exception`, which is undefined or falsey'}

The one below works particulary well with just prompting!!!


In [42]:
html_chunks = utils.html_chunker(table_rows, chunk_size=4)
html_input = utils.html_to_str(html_chunks[0])
output = html_to_json_ai(
    html_input,
    model="openchat/openchat-3.5-1210",
    json_format=json_format,
    api_key=config["TOGETHERAI_API_KEY"],
    url="https://api.together.xyz",
)

In [43]:
print(output.choices[0].message.content)

Based on the provided HTML and the specified JSON format, here's the converted JSON:
```json
{
    "items": [
        {
            "weekend_end_date": "January 8, 2023",
            "film": "Avatar: The Way of Water",
            "gross": "$45,838,986",
            "notes": "Black Panther: Wakanda Forever and Avatar: The Way of Water became the first two films to consecutively top the box office for four consecutive weekends each since The Hunger Games: Mockingjay – Part 2 and Star Wars: The Force Awakens in 2015 and 2016."
        },
        {
            "weekend_end_date": "January 15, 2023",
            "film": "Avatar: The Way of Water",
            "gross": "$32,824,684",
            "notes": "Black Panther: Wakanda Forever and Avatar: The Way of Water became the first two films to consecutively top the box office for five consecutive weekends each since Stakeout and Fatal Attraction in 1987."
        },
        {
            "weekend_end_date": "January 22, 2023",
            "